In [ ]:
!pip install albumentations opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 37.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 37.1 MB/s eta 0:00:00 0:00:01


In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import albumentations as A

# Define paths
input_image_folder = '/Users/hannahgillespie/Google Drive/My Drive/Imperial College London/Thesis/data/images'  # Replace with your image folder path
input_label_folder = '/Users/hannahgillespie/Google Drive/My Drive/Imperial College London/Thesis/data/labels'  # Replace with your label folder path
output_image_folder = '/Users/hannahgillespie/Google Drive/My Drive/Imperial College London/Thesis/data/images'  # Replace with the output folder path
output_label_folder = '/Users/hannahgillespie/Google Drive/My Drive/Imperial College London/Thesis/data/labels'  # Replace with the output folder path
text_file = '/Users/hannahgillespie/aod_detection/model/data_specs/synthetic.txt'

In [ ]:
# Define augmentation pipeline with individual steps
augmentations = {
    'GaussianBlur': A.GaussianBlur(blur_limit=(3, 7), p=1.0),
    'MotionBlur': A.MotionBlur(blur_limit=(3, 7), p=1.0),
    'HorizontalFlip': A.HorizontalFlip(p=1.0),
    'RandomRotate90': A.RandomRotate90(p=1.0),
    'HueSaturationValue': A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1.0),
    'RandomBrightnessContrast': A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),
    'GaussNoise': A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
    'Rotate': A.Rotate(limit=10, p=1.0),
    'Resize': A.Resize(480, 640, p=1.0)
}

# Function to clip and round bounding boxes
def clip_and_round_bboxes(bboxes, decimals=6):
    clipped_bboxes = []
    for bbox in bboxes:
        x_center, y_center, width, height = bbox
        # Clip to range [0, 1]
        x_center = np.clip(x_center, 0, 1)
        y_center = np.clip(y_center, 0, 1)
        width = np.clip(width, 0, 1)
        height = np.clip(height, 0, 1)
        # Round to avoid floating-point errors
        x_center = round(x_center, decimals)
        y_center = round(y_center, decimals)
        width = round(width, decimals)
        height = round(height, decimals)
        clipped_bboxes.append((x_center, y_center, width, height))
    return clipped_bboxes

# Function to draw bounding boxes on the image
def draw_bboxes(image, bboxes, class_labels):
    height, width = image.shape[:2]
    for bbox, label in zip(bboxes, class_labels):
        x_center, y_center, w, h = bbox
        # Convert normalized coordinates to absolute values
        x1 = int((x_center - w / 2) * width)
        y1 = int((y_center - h / 2) * height)
        x2 = int((x_center + w / 2) * width)
        y2 = int((y_center + h / 2) * height)
        # Draw the bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Ensure the text is drawn within the image boundaries
        text_x = max(x1, 0)
        text_y = max(y1 - 10, 10)  # Adjusted y-position to be above the box and within the image

        # Put the class label text with improved visibility
        cv2.putText(image, str(label), (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    return image

# Function to apply individual augmentations and save each result
def augment_image(image_path, label_path):
    # Read image
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Read YOLO label
    with open(label_path, 'r') as file:
        bboxes = []
        class_labels = []
        for line in file:
            class_id, x_center, y_center, w, h = map(float, line.split())
            bboxes.append([x_center, y_center, w, h])
            class_labels.append(int(class_id))

    # Apply and save each augmentation
    for aug_name, aug in augmentations.items():
        augmented = aug(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
        augmented_class_labels = augmented['class_labels']

        # Clip and round bounding boxes to be within the range [0, 1]
        augmented_bboxes = clip_and_round_bboxes(augmented_bboxes)

        # Draw bounding boxes on the augmented image
        #augmented_image_with_bboxes = draw_bboxes(augmented_image.copy(), augmented_bboxes, augmented_class_labels)

        # Display the augmented image
        #cv2.imshow(augmented_image_with_bboxes)
        #cv2.waitKey(1)  # Display each image for 500 milliseconds (adjust as needed)

        # Save augmented image with augmentation name
        output_image_path = os.path.join(output_image_folder, f'aug_{aug_name}_{os.path.basename(image_path)}')
        cv2.imwrite(output_image_path, augmented_image)

        # Save augmented labels
        output_label_path = os.path.join(output_label_folder, f'aug_{aug_name}_{os.path.basename(label_path)}')
        with open(output_label_path, 'w') as file:
            for bbox, class_id in zip(augmented_bboxes, augmented_class_labels):
                file.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

        # Save the augmented image name to the text file
        with open(text_file, 'a') as file:
            print(f'aug_{aug_name}_{os.path.basename(image_path)}\n')
            file.write(f'aug_{aug_name}_{os.path.basename(image_path)}\n')

# List all image filenames in the folder
image_filenames = [f for f in os.listdir(input_image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Perform augmentation on the selected images
for file_name in image_filenames:
    image_path = os.path.join(input_image_folder, file_name)
    label_path = os.path.join(input_label_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))
    if os.path.exists(label_path):
        augment_image(image_path, label_path)

# Close all OpenCV windows
cv2.destroyAllWindows()

KeyboardInterrupt: 